In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import timm
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from PIL import Image, ImageDraw, ImageFont
import json
from ultralytics import YOLO, YOLOWorld
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math
from transformers import get_cosine_schedule_with_warmup
from torchmetrics.wrappers import ClasswiseWrapper
from torchmetrics import MetricCollection
import albumentations as A
from IPython.display import clear_output
import cv2
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassPrecision,
    MulticlassRecall,
    MulticlassF1Score,
)
from pathlib import Path
from IPython.display import clear_output

## изначально нам даны 2 папки с фотками и 2 csv(train/test). Test идет без лейблов. У меня train_data.csv и test_data.csv - оригинальные. val_sp, train_sp, test_sp - поделенные сплиты из датасета train. Он отличается от train_data тем что лейблы - int. 

In [1]:
import pandas as pd

train_encoded = pd.read_csv("/probnik/purple/submission.csv")
print(len(train_encoded))
train_origin = pd.read_csv("/probnik/purple/metadata/train_data.csv")

1434


In [ ]:
train_encoded

In [ ]:
print(train_encoded[train_encoded["id"] == 37599275027]["target"].item())
print(train_origin[train_origin["id"] == 37599275027]["target"].item())

In [ ]:
a = train_origin["target"].value_counts()
plt.barh(y=a.keys(), width=a.values)

plt.show()

In [ ]:
all_densenet_models = timm.list_models("*resnext26*")
all_densenet_models

## ЗАГРУЗКА БЕЙЗЛАЙН МОДЕЛИ


In [ ]:
modelka = timm.create_model("resnext26ts", pretrained=True, num_classes=18)
checkpoint = torch.load(
    "/purple/exp/resnext26ts/checkpoints/epoch-008_step-405_f1macro-0.585192.ckpt",
    map_location="cpu",
)
state_dict = checkpoint["state_dict"]

from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace("model.", "") if k.startswith("model.") else k
    new_state_dict[name] = v

modelka.load_state_dict(new_state_dict)

## ПОИСК ДУБЛИКАТОВ

In [19]:
if torch.cuda.is_available():
    device = torch.device("cuda:7")
else:
    device = torch.device("cpu")
model = timm.create_model("vit_base_patch14_dinov2", pretrained=True, num_classes=0).to(
    device
)
device


class Datasetik(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        subdirs = ["train", "test", "valid"]

        for subdir in subdirs:
            folder_dir = os.path.join(self.data_dir, subdir, "images")
            for file in os.listdir(folder_dir):
                image_dir = os.path.join(folder_dir, file)
                if file.endswith(".jpg"):
                    self.images.append(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path)

        if self.transform:
            image = self.transform(img)

        return image, img_path


transform = transforms.Compose([transforms.Resize(518), transforms.ToTensor()])
transforms.ToTensor()
dataset = Datasetik(
    data_dir="/.",
    transform=transform,
)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

model.eval()

embed_list = []
image_paths = []
with torch.no_grad():
    for i, (images, paths) in enumerate(dataloader):
        images = images.to(device)
        output = model(images)
        embed_list.append(output.cpu().detach())
        image_paths.extend(paths)
        torch.cuda.empty_cache()
        print(f"epoch {i} ended")
embed_tensor = torch.cat(embed_list, dim=0)

# embed_tensor = torch.load('/probnik/purple/tensor.pt')
with open("image_paths.json", "r") as f:
    loaded_links = json.load(f)
dbscan = DBSCAN(eps=1, min_samples=2)
labels = dbscan.fit_predict(embed_tensor)
print(
    f"Количество кластеров (без учета шума):: {len(set(labels)) - (1 if -1 in labels else 0)}"
)
print(
    f"Количество точек шума: {list(labels).count(-1)}"
)  # фото которые кажутся уникальными

In [21]:
clustered_images = {}
for idx, label in enumerate(labels):
    if label != -1:
        if label not in clustered_images:
            clustered_images[label] = []
        clustered_images[label].append(loaded_links[idx])

In [ ]:
counetr = 0
for clster in clustered_images:
    counetr += len(clustered_images[clster])
print(counetr - len(clustered_images))  # количество фото для удаления

In [49]:
paths_to_delete = []

for claster in clustered_images:
    paths_to_delete.extend(clustered_images[claster][1:])

## Кроп изображений на основе обьекта в csv


In [66]:
model = YOLOWorld("yolov8l-worldv2.pt").to("cuda:7")
DATA_DIR = "/probnik/purple/train_data"
DATA_CSV = pd.read_csv("/probnik/purple/metadata/train_data.csv")
categories = {
    "столы": ["table", "desk", "dining table", "coffee table"],
    "стулья": ["chair", "armchair", "dining chair", "office chair"],
    "сумки": ["suitcase", "backpack", "bag", "handbag", "wallet"],
    "одежда для девочек": [
        "kids clothes(female)",
        "female clothes",
        "woman clothes",
        "clothes",
    ],
}
output_dir = Path("Cropped_images_2")
output_dir.mkdir(exist_ok=True)

for filename in tqdm(os.listdir(DATA_DIR)):
    img_path = os.path.join(DATA_DIR, filename)
    img_name = Path(img_path).stem
    category = DATA_CSV.loc[DATA_CSV["id"] == int(img_name)]["category"].item()
    model.set_classes(categories[category])
    results = model.predict(img_path)

    clear_output()

    image = cv2.imread(img_path)

    dir_save = output_dir / str(img_name)
    dir_save.mkdir(exist_ok=True)

    if len(results[0].boxes.data) > 0:
        for i, coords in enumerate(results[0].boxes.data):
            coords = coords.cpu().numpy()
            x_min = int(coords[0])
            y_min = int(coords[1])
            x_max = int(coords[2])
            y_max = int(coords[3])
            cropped = image[y_min:y_max, x_min:x_max]
            cv2.imwrite(
                str(dir_save / f"crop_{i}.jpg"),
                cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR),
            )
    else:
        cv2.imwrite(str(dir_save / "orig.jpg"), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

## Трейн луп


In [35]:
train_csv = "//probnik/purple/metadata/train_sp.csv"
test_csv = "/probnik/purple/metadata/test_sp.csv"
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)


class CustomDatasetAlt(Dataset):
    def __init__(self, root_dir, label_df, transform=None):
        self.root_dir = Path(root_dir)
        self.label_df = label_df
        self.transform = transform
        self.samples = []

        for subdir in self.root_dir.iterdir():
            if subdir.is_dir():
                current_id = int(subdir.name)
                row = self.label_df[self.label_df["id"] == current_id]
                if row.empty:
                    continue
                label = int(row["target"].values[0])
                for file in subdir.iterdir():
                    if file.suffix.lower() in [".png", ".jpg", ".jpeg"]:
                        self.samples.append((str(file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        img_path, label = self.samples[index]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


transform_alt = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

root_dir = "/probnik/purple/Cropped_images_2"

train_dataset_alt = CustomDatasetAlt(
    root_dir=root_dir, label_df=train_df, transform=transform_alt
)
train_loader_alt = DataLoader(
    train_dataset_alt, batch_size=2048, shuffle=True, num_workers=4
)

test_dataset_alt = CustomDatasetAlt(
    root_dir=root_dir, label_df=test_df, transform=transform_alt
)
test_loader_alt = DataLoader(test_dataset_alt, batch_size=2048, num_workers=4)

In [ ]:
class TorchModel(nn.Module):
    def __init__(self, encoder, output):
        super(TorchModel, self).__init__()
        self.model = timm.create_model(encoder, pretrained=True, num_classes=output)

        for param in self.model.stem.parameters():
            param.requires_grad = False
        # for param in self.model.stages.parameters():
        #     param.requires_grad = False
        for param in self.model.stages[:3].parameters():
            param.requires_grad = False
        for param in self.model.stages[3][0].parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.model(x)


def train_model(train_loader, val_loader, num_epochs=20):
    encoder = "resnext26ts"
    save_path = os.path.join("./exp", encoder + "FT_cropp")
    os.makedirs(save_path, exist_ok=True)

    labels = [
        "belyi",
        "bezhevyi",
        "biryuzovyi",
        "bordovyi",
        "chernyi",
        "fioletovyi",
        "goluboi",
        "korichnevyi",
        "krasnyi",
        "oranzhevyi",
        "raznocvetnyi",
        "rozovyi",
        "serebristyi",
        "seryi",
        "sinii",
        "zelenyi",
        "zheltyi",
        "zolotoi",
    ]

    device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
    model = TorchModel(encoder, 18).to(device)
    criterion = nn.CrossEntropyLoss()

    train_metrics = MetricCollection(
        {
            "multiclassAccuracy": ClasswiseWrapper(
                MulticlassAccuracy(num_classes=18, average=None), labels=labels
            ),
            "multiclassPrecision": ClasswiseWrapper(
                MulticlassPrecision(num_classes=18, average=None), labels=labels
            ),
            "multiclassRecall": ClasswiseWrapper(
                MulticlassRecall(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1score": ClasswiseWrapper(
                MulticlassF1Score(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1scoreMacro": MulticlassF1Score(
                num_classes=18, average="macro"
            ),
            "multiclassPrecision_macro": MulticlassPrecision(
                num_classes=18, average="macro"
            ),
            "multiclassRecall_macro": MulticlassRecall(num_classes=18, average="macro"),
        }
    ).to(device)

    val_metrics = MetricCollection(
        {
            "multiclassAccuracy": ClasswiseWrapper(
                MulticlassAccuracy(num_classes=18, average=None), labels=labels
            ),
            "multiclassPrecision": ClasswiseWrapper(
                MulticlassPrecision(num_classes=18, average=None), labels=labels
            ),
            "multiclassRecall": ClasswiseWrapper(
                MulticlassRecall(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1score": ClasswiseWrapper(
                MulticlassF1Score(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1scoreMacro": MulticlassF1Score(
                num_classes=18, average="macro"
            ),
            "multiclassPrecision_macro": MulticlassPrecision(
                num_classes=18, average="macro"
            ),
            "multiclassRecall_macro": MulticlassRecall(num_classes=18, average="macro"),
        }
    ).to(device)

    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=0.0002,
        weight_decay=0.0001,
    )

    num_training_steps = math.ceil(len(train_loader))  # количество шагов за эпоху
    total_steps = num_training_steps * num_epochs
    scheduler = get_cosine_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )

    train_loss_history = []
    val_loss_history = []
    train_macroF1_history = []
    val_macroF1_history = []
    train_macroPrecision_history = []
    val_macroPrecision_history = []
    train_macroRecall_history = []
    val_macroRecall_history = []

    best_pr_macro = -float("inf")
    plt.ion()

    for epoch in range(num_epochs):
        model.train()
        train_metrics.reset()
        epoch_train_loss = 0.0
        train_samples = 0

        pbar = tqdm(
            train_loader, desc=f"Train Epoch {epoch+1}/{num_epochs}", leave=False
        )
        for batch in pbar:
            x, y = batch
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_train_loss += loss.item() * x.size(0)
            train_samples += x.size(0)

            preds = outputs.argmax(dim=1)
            train_metrics.update(preds, y)

        avg_train_loss = epoch_train_loss / train_samples
        train_values = train_metrics.compute()

        clear_output()
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")
        for metric_name, value in train_values.items():
            print(f"Train {metric_name}: {value}")

        model.eval()
        val_metrics.reset()
        epoch_val_loss = 0.0
        val_samples = 0

        with torch.no_grad():
            for batch in val_loader:
                x, y = batch
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                loss = criterion(outputs, y)
                epoch_val_loss += loss.item() * x.size(0)
                val_samples += x.size(0)

                preds = outputs.argmax(dim=1)
                val_metrics.update(preds, y)

        avg_val_loss = epoch_val_loss / val_samples
        val_values = val_metrics.compute()

        print(f"Epoch {epoch+1}/{num_epochs} - Val Loss: {avg_val_loss:.4f}")
        for metric_name, value in val_values.items():
            print(f"Val {metric_name}: {value}")

        train_loss_history.append(avg_train_loss)
        val_loss_history.append(avg_val_loss)

        train_macroF1 = train_values.get("multiclassF1scoreMacro")
        val_macroF1 = val_values.get("multiclassF1scoreMacro")
        train_macroPrecision = train_values.get("multiclassPrecision_macro")
        val_macroPrecision = val_values.get("multiclassPrecision_macro")
        train_macroRecall = train_values.get("multiclassRecall_macro")
        val_macroRecall = val_values.get("multiclassRecall_macro")

        train_macroF1 = (
            train_macroF1.item()
            if isinstance(train_macroF1, torch.Tensor)
            else train_macroF1
        )
        val_macroF1 = (
            val_macroF1.item() if isinstance(val_macroF1, torch.Tensor) else val_macroF1
        )
        train_macroPrecision = (
            train_macroPrecision.item()
            if isinstance(train_macroPrecision, torch.Tensor)
            else train_macroPrecision
        )
        val_macroPrecision = (
            val_macroPrecision.item()
            if isinstance(val_macroPrecision, torch.Tensor)
            else val_macroPrecision
        )
        train_macroRecall = (
            train_macroRecall.item()
            if isinstance(train_macroRecall, torch.Tensor)
            else train_macroRecall
        )
        val_macroRecall = (
            val_macroRecall.item()
            if isinstance(val_macroRecall, torch.Tensor)
            else val_macroRecall
        )

        if val_macroF1 > best_pr_macro:
            best_pr_macro = val_macroF1
            torch.save(model.state_dict(), os.path.join(save_path, "bestmodel.pth"))

        train_macroF1_history.append(train_macroF1)
        val_macroF1_history.append(val_macroF1)
        train_macroPrecision_history.append(train_macroPrecision)
        val_macroPrecision_history.append(val_macroPrecision)
        train_macroRecall_history.append(train_macroRecall)
        val_macroRecall_history.append(val_macroRecall)

        plt.figure(1)
        plt.gcf().set_size_inches(12, 10)
        plt.clf()

        plt.subplot(3, 1, 1)
        plt.plot(train_loss_history, label="Train Loss", marker="o")
        plt.plot(val_loss_history, label="Val Loss", marker="o")
        plt.legend()
        plt.title("Loss per Epoch")

        plt.subplot(3, 1, 2)
        plt.plot(train_macroF1_history, label="Train Macro F1", marker="o")
        plt.plot(val_macroF1_history, label="Val Macro F1", marker="o")
        plt.legend()
        plt.title("Macro F1 Score per Epoch")

        plt.subplot(3, 1, 3)
        plt.plot(
            train_macroPrecision_history, label="Train Macro Precision", marker="o"
        )
        plt.plot(val_macroPrecision_history, label="Val Macro Precision", marker="o")
        plt.plot(train_macroRecall_history, label="Train Macro Recall", marker="o")
        plt.plot(val_macroRecall_history, label="Val Macro Recall", marker="o")
        plt.legend()
        plt.title("Macro Precision & Recall per Epoch")

        plt.pause(0.01)
        plt.draw()

    torch.save(model.state_dict(), os.path.join(save_path, "model_final.pth"))

    plt.ioff()
    plt.show()


train_model(train_loader=train_loader_alt, val_loader=test_loader_alt, num_epochs=20)

## Поиск доминирующего пикселя по фотографии, поиск ближайшего в rgb. Работает СЛИШКОМ долго

In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

COLOR_PALETTE = {
    "belyi": (255, 255, 255),
    "bezhevyi": (245, 245, 220),
    "biryuzovyi": (64, 224, 208),
    "bordovyi": (128, 0, 0),
    "chernyi": (0, 0, 0),
    "fioletovyi": (238, 85, 238),
    "goluboi": (173, 216, 230),
    "korichnevyi": (150, 75, 0),
    "krasnyi": (255, 0, 0),
    "oranzhevyi": (255, 165, 0),
    "raznocvetnyi": (128, 128, 128),
    "rozovyi": (255, 192, 203),
    "serebristyi": (192, 192, 192),
    "seryi": (128, 128, 128),
    "sinii": (0, 0, 255),
    "zelenyi": (0, 128, 0),
    "zheltyi": (255, 255, 0),
    "zolotoi": (255, 215, 0),
}


def closest_color(rgb, palette):
    """Находит ближайший цвет из палитры"""
    colors = np.array(list(palette.values()))
    color = np.array(rgb)
    distances = np.sqrt(np.sum((colors - color) ** 2, axis=1))
    index = np.argmin(distances)
    return list(palette.keys())[index]


def get_dominant_color(image_path, k=3):
    """Определяет доминирующий цвет изображения с оптимизациями"""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Не удалось загрузить изображение")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (200, 200))

    pixels = image.reshape(-1, 3)
    if pixels.shape[0] > 1000:
        indices = np.random.choice(pixels.shape[0], 1000, replace=False)
        pixels = pixels[indices]

    kmeans = KMeans(n_clusters=k, n_init=3, max_iter=100)
    kmeans.fit(pixels)

    counts = np.bincount(kmeans.labels_)
    dominant = kmeans.cluster_centers_[np.argmax(counts)]
    return dominant.astype(int)


def process_directory(dir_path):
    """Обрабатывает одну директорию"""
    colors = []
    for fname in os.listdir(dir_path):
        if fname.lower().endswith((".png", ".jpg", ".jpeg")):
            try:
                color = get_dominant_color(os.path.join(dir_path, fname))
                colors.append(color)
            except:
                continue

    if not colors:
        return None

    avg_color = np.mean(colors, axis=0).astype(int)
    return closest_color(avg_color, COLOR_PALETTE)


def main(root_dir, output_file="results.csv"):
    """Основная функция"""
    directories = [os.path.join(root_dir, d) for d in os.listdir(root_dir)]

    with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor, open(
        output_file, "w"
    ) as f:

        f.write("Directory,DominantColor\n")
        results = executor.map(process_directory, directories)

        total_dirs = len(os.listdir(root_dir))
        for dir_name, color in tqdm(
            zip(os.listdir(root_dir), results), total=total_dirs
        ):
            if color:
                f.write(f"{dir_name},{color}\n")


main("/probnik/purple/Cropped_images_2")

In [ ]:
train_origin.loc[train_origin["category"] == "одежда для девочек"]["id"].head(50)

In [ ]:
train_origin.loc[train_origin["id"] == 25497098113]

In [ ]:
imgsd = Image.open("/probnik/purple/train_data/29097043010.jpg")

imgsd

# Визуализация повторяющихся фото

In [ ]:
for cluster_id in sorted(clustered_images.keys())[:30]:  # первые 30 кластеров
    image_list = clustered_images[cluster_id]
    num_images = len(image_list)

    fig, ax = plt.subplots(1, num_images, figsize=(20, 10))

    if num_images == 1:
        ax = [ax]

    for i, image_path in enumerate(image_list):
        try:
            img = Image.open(image_path)
            ax[i].imshow(img)
            ax[i].axis("off")
            ax[i].set_title(f"Cluster {cluster_id}\nImage {i}")
        except Exception as e:
            print(f"Error loading {image_path}: {e}")

    plt.tight_layout()
    plt.show()

## датасет
 



In [38]:
# def train_transforms(probs=0.25):
#     return A.Compose(
#         [
#             A.Resize(224, 224),
#             A.OneOf([
#                 A.HorizontalFlip(p=1),
#                 A.VerticalFlip(p=1),
#             ], p=probs),
#             A.Normalize(),
#             ToTensorV2()
#         ]
#     )


def train_transforms(probs=0.25):
    return A.Compose(
        [
            A.Resize(224, 224),
            A.OneOf(
                [
                    A.HorizontalFlip(p=1),
                    A.VerticalFlip(p=1),
                ],
                p=probs,
            ),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(
                shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5
            ),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.Normalize(),
            ToTensorV2(),
        ]
    )


def val_transform():
    return A.Compose([A.Resize(224, 224), A.Normalize(), ToTensorV2()])


class TorchDataset(Dataset):
    def __init__(self, csv_file, img_dir, mode="train"):
        self.images = []
        self.img_dir = img_dir
        self.mode = mode
        self.__read_csv(csv_file)
        self.__init_transforms(aug_prob=0.5)

    def __read_csv(self, csv_file):
        with open(csv_file, "r") as file:
            for i, line in enumerate(file.readlines()):
                if i == 0:
                    continue
                image, _, label = line.strip().split(",")
                self.images.append([image, int(label)])

    def __init_transforms(self, aug_prob):
        self.transforms = (
            train_transforms(aug_prob) if self.mode == "train" else val_transform()
        )

    def __len__(self):
        return len(self.images)

    def load_sample(self, idx):
        image_path, label = self.images[idx]

        if not os.path.exists(os.path.join(self.img_dir, image_path + ".jpg")):
            raise ValueError(f"{os.path.join(self.img_dir, image_path)} doesn't exist")
        image = cv2.imread(os.path.join(self.img_dir, image_path + ".jpg"))
        return image, label

    def __getitem__(self, idx):
        image, label = self.load_sample(idx)
        if self.transforms is not None:
            image = self.transforms(image=image)["image"]
        return image, torch.as_tensor(label)


def make_loader(csv_file, img_dir, mode="train"):
    dataset = TorchDataset(csv_file, img_dir, mode=mode)
    dataloader = DataLoader(
        dataset, shuffle=(mode == "train"), batch_size=128, num_workers=4
    )
    return dataloader, dataset

In [ ]:
train_loader, train_dataset = make_loader(
    "/probnik/purple/metadata/train_sp.csv",
    "/probnik/purple/train_data",
    mode="train",
)
test_loader, test_dataset = make_loader(
    "/probnik/purple/metadata/test_sp.csv",
    "/probnik/purple/train_data",
    mode="test",
)
val_loader, val_dataset = make_loader(
    "/probnik/purple/metadata/val_sp.csv",
    "/probnik/purple/train_data",
    mode="val",
)

In [ ]:
print(len(test_dataset), len(train_dataset), len(val_dataset))

## Модель





In [41]:
class TorchModel(nn.Module):
    def __init__(self, encoder, output):
        super(TorchModel, self).__init__()
        self.model = timm.create_model(encoder, pretrained=True, num_classes=output)

        # for param in self.model.stem.parameters():
        #     param.requires_grad = False
        # # for param in self.model.stages.parameters():
        # #     param.requires_grad = False
        # for param in self.model.stages[:3].parameters():
        #     param.requires_grad = False
        # for param in self.model.stages[3][0].parameters():
        #     param.requires_grad = False

    def forward(self, x):
        return self.model(x)

In [ ]:
model = TorchModel("resnet50", 18)


# Общее количество параметров:
total_params = sum(p.numel() for p in model.parameters())

# Количество замороженных (необучаемых) параметров:
frozen_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)

# Количество обучаемых параметров:
trainable_params = total_params - frozen_params

print("Всего параметров:", total_params)
print("Замороженных параметров:", frozen_params)
print("Обучаемых параметров:", trainable_params)

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()

In [ ]:
def train_model(train_loader, val_loader, num_epochs=20):
    encoder = "resnet50"
    save_path = os.path.join("./exp", encoder + "focal_loss")
    os.makedirs(save_path, exist_ok=True)
    labels = [
        "belyi",
        "bezhevyi",
        "biryuzovyi",
        "bordovyi",
        "chernyi",
        "fioletovyi",
        "goluboi",
        "korichnevyi",
        "krasnyi",
        "oranzhevyi",
        "raznocvetnyi",
        "rozovyi",
        "serebristyi",
        "seryi",
        "sinii",
        "zelenyi",
        "zheltyi",
        "zolotoi",
    ]

    device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
    model = TorchModel(encoder, 18).to(device)
    criterion = FocalLoss()

    train_metrics = MetricCollection(
        {
            "multiclassAccuracy": ClasswiseWrapper(
                MulticlassAccuracy(num_classes=18, average=None), labels=labels
            ),
            "multiclassPrecision": ClasswiseWrapper(
                MulticlassPrecision(num_classes=18, average=None), labels=labels
            ),
            "multiclassRecall": ClasswiseWrapper(
                MulticlassRecall(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1score": ClasswiseWrapper(
                MulticlassF1Score(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1scoreMacro": MulticlassF1Score(
                num_classes=18, average="macro"
            ),
            "multiclassPrecision_macro": MulticlassPrecision(
                num_classes=18, average="macro"
            ),
            "multiclassRecall_macro": MulticlassRecall(num_classes=18, average="macro"),
        }
    ).to(device)

    val_metrics = MetricCollection(
        {
            "multiclassAccuracy": ClasswiseWrapper(
                MulticlassAccuracy(num_classes=18, average=None), labels=labels
            ),
            "multiclassPrecision": ClasswiseWrapper(
                MulticlassPrecision(num_classes=18, average=None), labels=labels
            ),
            "multiclassRecall": ClasswiseWrapper(
                MulticlassRecall(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1score": ClasswiseWrapper(
                MulticlassF1Score(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1scoreMacro": MulticlassF1Score(
                num_classes=18, average="macro"
            ),
            "multiclassPrecision_macro": MulticlassPrecision(
                num_classes=18, average="macro"
            ),
            "multiclassRecall_macro": MulticlassRecall(num_classes=18, average="macro"),
        }
    ).to(device)

    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=0.0002,
        weight_decay=0.0001,
    )  # бубугага

    num_training_steps = math.ceil(len(train_loader) / 1)  #  2
    total_steps = num_training_steps * num_epochs
    scheduler = get_cosine_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )

    train_loss_history = []
    val_loss_history = []
    train_macroF1_history = []
    val_macroF1_history = []
    train_macroPrecision_history = []
    val_macroPrecision_history = []
    train_macroRecall_history = []
    val_macroRecall_history = []

    best_pr_macro = -float("inf")
    plt.ion()

    for epoch in range(num_epochs):
        model.train()
        train_metrics.reset()
        epoch_train_loss = 0.0
        train_samples = 0

        pbar = tqdm(
            train_loader, desc=f"Train Epoch {epoch+1}/{num_epochs}", leave=False
        )
        for batch in pbar:
            x, y = batch
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_train_loss += loss.item() * x.size(0)
            train_samples += x.size(0)

            preds = outputs.argmax(dim=1)
            train_metrics.update(preds, y)

        avg_train_loss = epoch_train_loss / train_samples
        train_values = train_metrics.compute()

        clear_output(wait=True)
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")
        for metric_name, value in train_values.items():
            print(f"Train {metric_name}: {value}")

        model.eval()
        val_metrics.reset()
        epoch_val_loss = 0.0
        val_samples = 0

        with torch.no_grad():
            for batch in val_loader:
                x, y = batch
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                loss = criterion(outputs, y)
                epoch_val_loss += loss.item() * x.size(0)
                val_samples += x.size(0)

                preds = outputs.argmax(dim=1)
                val_metrics.update(preds, y)

        avg_val_loss = epoch_val_loss / val_samples
        val_values = val_metrics.compute()

        print(f"Epoch {epoch+1}/{num_epochs} - Val Loss: {avg_val_loss:.4f}")
        for metric_name, value in val_values.items():
            print(f"Val {metric_name}: {value}")

        train_loss_history.append(avg_train_loss)
        val_loss_history.append(avg_val_loss)

        train_macroF1 = train_values.get("multiclassF1scoreMacro")
        val_macroF1 = val_values.get("multiclassF1scoreMacro")
        train_macroPrecision = train_values.get("multiclassPrecision_macro")
        val_macroPrecision = val_values.get("multiclassPrecision_macro")
        train_macroRecall = train_values.get("multiclassRecall_macro")
        val_macroRecall = val_values.get("multiclassRecall_macro")

        train_macroF1 = (
            train_macroF1.item()
            if isinstance(train_macroF1, torch.Tensor)
            else train_macroF1
        )
        val_macroF1 = (
            val_macroF1.item() if isinstance(val_macroF1, torch.Tensor) else val_macroF1
        )
        train_macroPrecision = (
            train_macroPrecision.item()
            if isinstance(train_macroPrecision, torch.Tensor)
            else train_macroPrecision
        )
        val_macroPrecision = (
            val_macroPrecision.item()
            if isinstance(val_macroPrecision, torch.Tensor)
            else val_macroPrecision
        )
        train_macroRecall = (
            train_macroRecall.item()
            if isinstance(train_macroRecall, torch.Tensor)
            else train_macroRecall
        )
        val_macroRecall = (
            val_macroRecall.item()
            if isinstance(val_macroRecall, torch.Tensor)
            else val_macroRecall
        )

        if val_macroF1 > best_pr_macro:
            best_pr_macro = val_macroF1
            torch.save(
                model.state_dict(),
                os.path.join(
                    save_path, f"bestmodel_macrof1:{val_macroF1}_epoch:{epoch+1}.pth"
                ),
            )

        train_macroF1_history.append(train_macroF1)
        val_macroF1_history.append(val_macroF1)
        train_macroPrecision_history.append(train_macroPrecision)
        val_macroPrecision_history.append(val_macroPrecision)
        train_macroRecall_history.append(train_macroRecall)
        val_macroRecall_history.append(val_macroRecall)

        plt.figure(1)
        plt.gcf().set_size_inches(12, 10)
        plt.clf()

        plt.subplot(3, 1, 1)
        plt.plot(train_loss_history, label="Train Loss", marker="o")
        plt.plot(val_loss_history, label="Val Loss", marker="o")
        plt.legend()
        plt.title("Loss per Epoch")

        plt.subplot(3, 1, 2)
        plt.plot(train_macroF1_history, label="Train Macro F1", marker="o")
        plt.plot(val_macroF1_history, label="Val Macro F1", marker="o")
        plt.legend()
        plt.title("Macro F1 Score per Epoch")

        plt.subplot(3, 1, 3)
        plt.plot(
            train_macroPrecision_history, label="Train Macro Precision", marker="o"
        )
        plt.plot(val_macroPrecision_history, label="Val Macro Precision", marker="o")
        plt.plot(train_macroRecall_history, label="Train Macro Recall", marker="o")
        plt.plot(val_macroRecall_history, label="Val Macro Recall", marker="o")
        plt.legend()
        plt.title("Macro Precision & Recall per Epoch")

        plt.pause(0.01)
        plt.draw()

        # torch.save(model.state_dict(), f"checkpoint_epoch_{epoch+1}.pth")

    torch.save(model.state_dict(), os.path.join(save_path, "model_final.pth"))
    plt.savefig(os.path.join(save_path, "loss.png"))
    plt.ioff()
    plt.show()


train_model(train_loader=train_loader, val_loader=val_loader)

In [ ]:
def test_model(test_loader, weight_path):
    labels = [
        "belyi",
        "bezhevyi",
        "biryuzovyi",
        "bordovyi",
        "chernyi",
        "fioletovyi",
        "goluboi",
        "korichnevyi",
        "krasnyi",
        "oranzhevyi",
        "raznocvetnyi",
        "rozovyi",
        "serebristyi",
        "seryi",
        "sinii",
        "zelenyi",
        "zheltyi",
        "zolotoi",
    ]

    device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

    # Создаем модель и загружаем веса
    model = TorchModel("resnext26ts", 18).to(device)
    state_dict = torch.load(weight_path, map_location=device)
    model.load_state_dict(state_dict, strict=True)

    test_metrics = MetricCollection(
        {
            "multiclassPrecision": ClasswiseWrapper(
                MulticlassPrecision(num_classes=18, average=None), labels=labels
            ),
            "multiclassRecall": ClasswiseWrapper(
                MulticlassRecall(num_classes=18, average=None), labels=labels
            ),
            "multiclassF1score": ClasswiseWrapper(
                MulticlassF1Score(num_classes=18, average=None), labels=labels
            ),
            "multiclassAccuracy": ClasswiseWrapper(
                MulticlassAccuracy(num_classes=18, average=None), labels=labels
            ),
            "multiclassPrecision_macro": MulticlassPrecision(
                num_classes=18, average="macro"
            ),
            "multiclassRecall_macro": MulticlassRecall(num_classes=18, average="macro"),
            "multiclassF1score_macro": MulticlassF1Score(
                num_classes=18, average="macro"
            ),
            "multiclassAccuracy_macro": MulticlassAccuracy(
                num_classes=18, average="macro"
            ),
        }
    ).to(device)
    test_metrics.reset()

    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            preds = outputs.argmax(dim=1)
            test_metrics.update(preds, y)

    test_values = test_metrics.compute()

    data = [(k, v.item()) for k, v in test_values.items()]

    # Создаем DataFrame
    df = pd.DataFrame(data, columns=["metric", "value"])

    name = weight_path.split("/")[-2]
    df.to_csv(os.path.join("./exp", name, "torch_metrics.csv"), index=False)

    print("Test Metrics:")
    for metric_name, value in test_values.items():
        print(f"{metric_name}: {value}")
    return


test_model(
    test_loader=test_loader,
    weight_path="/probnik/purple/exp/resnext26tsmore augs/bestmodel.pth",
)